In [1]:
!pip install tigramite==5.2.5.8 numba==0.58.1 numpy==1.26.4 llvmlite==0.41.1


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import numpy as np
import pickle
from safetensors.torch import load_file

import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.lpcmci import LPCMCI
from tigramite.independence_tests.parcorr import ParCorr

import matplotlib.pyplot as plt

In [10]:
dist_type = "gumbel"
window_sizes_to_run = [450, 300, 150]
seeds_to_run = [42, 123, 2024]
causal_algorithms = {'PCMCI': PCMCI, 'LPCMCI': LPCMCI}
all_causal_results_gumbel_noisy = {}

output_results_dir = f"../result/causal_results_{dist_type}_parcorr"
output_images_dir = f"../result/causal_graphs_{dist_type}_parcorr"
os.makedirs(output_results_dir, exist_ok=True)
os.makedirs(output_images_dir, exist_ok=True)

In [11]:
activities = ['Walk', 'Run', 'Jump', 'Sitting', 'Empty Room']

def load_latent_data(latent_space_dir, window_size, seed, dist_type, activity_name):
    all_features = []
    num_antennas = 4
    for antenna_idx in range(num_antennas):
        dist_tag = dist_type[:4]
        filename = f"{dist_type[:4]}_win{window_size}_ant{antenna_idx}_seed{seed}.safetensors"
        path = os.path.join(latent_space_dir, filename)
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")
        loaded_tensors = load_file(path)
        activity_index = activities.index(activity_name)
        labels_np = loaded_tensors["labels"].numpy()
        indices = np.where(labels_np == activity_index)[0]
        features = loaded_tensors["latent_space"].numpy()[indices]
        all_features.append(features)
    fused_features = np.concatenate(all_features, axis=1)
    return fused_features

def prepare_gumbel_data_noisy(one_hot_vectors, noise_level=0.01):
    
    print(f"  > Converting one-hot to continuous by adding noise (std={noise_level})...")
    noise = np.random.normal(0, noise_level, one_hot_vectors.shape)
    return one_hot_vectors + noise

In [12]:
for window_size in window_sizes_to_run:
    all_causal_results_gumbel_noisy[window_size] = {}
    for seed in seeds_to_run:
        all_causal_results_gumbel_noisy[window_size][seed] = {}
        for activity in activities:
            print(f"--- Processing: Gumbel (ParCorr-Noisy), Win={window_size}, Seed={seed}, Activity={activity} ---")
            try:
                latent_space_dir = f"../dataset/latent_space_dataset_{dist_type}"
                one_hot_vectors = load_latent_data(latent_space_dir, window_size, seed, dist_type, activity)
                
                if one_hot_vectors.shape[0] < 20: continue
                
                continuous_data = prepare_gumbel_data_noisy(one_hot_vectors)
                
                num_features = continuous_data.shape[1]
                proposition_names = [f'F_{i}' for i in range(num_features)]
                
                dataframe = pp.DataFrame(continuous_data, var_names=proposition_names)
                cond_ind_test = ParCorr() 
                
                results_per_algo = {}
                for algo_name, algo_class in causal_algorithms.items():
                    print(f"  > Running {algo_name}...")
                    causal_discovery = algo_class(dataframe=dataframe, cond_ind_test=cond_ind_test, verbosity=0)
                    if algo_name == 'PCMCI':
                        results = causal_discovery.run_pcmci(tau_max=3, pc_alpha=0.01)
                    else:
                        results = causal_discovery.run_lpcmci(tau_min=1, tau_max=3, pc_alpha=0.01)
                    results_per_algo[algo_name] = results
                    
                    fig, ax = tp.plot_time_series_graph(figsize=(15, 12), val_matrix=results['val_matrix'], graph=results['graph'], var_names=proposition_names, link_colorbar_label='ParCorr Coeff.', cmap_edges="viridis")
                    fig.suptitle(f'Causal Graph ({algo_name}) for {activity}\nGumbel (ParCorr-Noisy), win={window_size}, seed={seed}', y=1.0)
                    image_filename = f"gumb_win{window_size}_seed{seed}_{activity}_{algo_name}_parcorr_noisy.png"
                    fig.savefig(os.path.join(output_images_dir, image_filename), bbox_inches='tight')
                    plt.close(fig)
                
                all_causal_results_gumbel_noisy[window_size][seed][activity] = results_per_algo
            except Exception as e:
                print(f"  > An error occurred: {e}\n")
                
results_filepath = os.path.join(output_results_dir, 'all_causal_results_gumbel_noisy.pkl')
with open(results_filepath, 'wb') as f:
    pickle.dump(all_causal_results_gumbel_noisy, f)
print(f"\n--- Gumbel (ParCorr-Noisy) experiments processed! Results saved to: {results_filepath} ---")

--- Processing: Gumbel (ParCorr-Noisy), Win=450, Seed=42, Activity=Walk ---
  > Converting one-hot to continuous by adding noise (std=0.01)...
  > Running PCMCI...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Momo\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Momo\AppData\Local\Temp\ipykernel_22420\383210299.py", line 27, in <module>
    results = causal_discovery.run_pcmci(tau_max=3, pc_alpha=0.01)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Momo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tigramite\pcmci.py", line 1930, in run_pcmci
    results = self.run_mci(link_assumptions=link_assumptions,
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Momo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tigramite\pcmci.py", line 1242, in run_mci
    return self._run_mci_or_variants(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Momo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tigramite\pcmci.py", line 108